In [1]:
                                                                                                                                                                                                                                                          import pandas as pd
import numpy as np
import string
import nltk
import spacy

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from utils import cosine_sim ## This only for testing
from utils import get_upper_triangular_values, threshold_transform

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [3]:
df_iem = pd.read_excel('newdata/VT-IEM.xlsx')
df_igk = pd.read_excel('newdata/VT-IGK.xlsx')
df_ika = pd.read_excel('newdata/VT-IKA.xlsx')
df_ikd = pd.read_excel('newdata/VT-IKD.xlsx')
df_inu = pd.read_excel('newdata/VT-INU.xlsx')
df_ihv = pd.read_excel('newdata/VT-IHV.xlsx')

In [4]:
df_iem.head(3)

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral FC,Deferral Parameter,Deferral Value,Deferral Calendar Value,Deferral Calendar,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center
0,No,MIREP,HMV21/000097/1021/224,24,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/224,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN
1,No,MIREP,HMV21/000097/1021/395,24,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,HMV21/000097/1021/395,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN
2,No,MIREP,HMV21/000097/1021/790,52,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,HMV21/000097/1021/790,Close,Closed,200133-01-1,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN


In [5]:
# Download NLTK Resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess_text(text: str, preserve_symbols=[], words_to_remove=['DURING', 'INSPECTION', 'OBSERVED']) -> str:
    '''
    This function performs text preprocessing and returns processed text. 
    It will also accept a list of symbols to preserve.
    Input: text
    Output: text
    '''
    #Newly added code to address an NaN float error
    if isinstance(text, float) and np.isnan(text):  
        return ''  
    
    # Define symbols to preserve
    preserve_symbols = set(preserve_symbols)

    for word in words_to_remove:
        text = text.replace(word, ' ')
    
    # Remove punctuation, excluding specified symbols
    custom_translation = str.maketrans('', '', ''.join(set(string.punctuation) - preserve_symbols))
    text = text.translate(custom_translation)
    return text


def tokenization(preprocessed_text: str) -> list:
    '''
    This function performs text tokenization and returns a list of tokens.
    Input: text
    Output: list of tokens
    '''
    sentences = sent_tokenize(preprocessed_text)
    preprocessed_tokens = []

    # Tokenization and stopword removal
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token.lower() not in stop_words]
        
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        preprocessed_tokens.append(tokens)

    return preprocessed_tokens


def calculate_embeddings(preprocessed_tokens: list) -> list:
    '''
    This function performs text embedding and returns a list of embeddings.
    Input: list of tokens
    Output: list of embeddings
    '''
    # Embedding
    embeddings = []
    for token in preprocessed_tokens:
        sentence = ' '.join(token)
        doc = nlp(sentence)
        sentence_embedding = doc.vector
        embeddings.append(sentence_embedding)

    return embeddings


def main(text: str, preserve_symbols=['-', '/']) -> list:
    '''
    This function performs text preprocessing, tokenization, embedding and returns a list of embeddings.
    Input: text
    Output: list of embeddings
    '''
    preprocessed_text = preprocess_text(text, preserve_symbols)
    preprocessed_tokens = tokenization(preprocessed_text)
    embeddings = calculate_embeddings(preprocessed_tokens)

    return embeddings

In [7]:
#IEM

In [8]:
grouped_df_Description = df_iem.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_iem.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_iem['description_correction'] = df_iem['Description'] +' '+df_iem['Corrective Action']
grouped_df_description_correction = df_iem.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [9]:
grouped_df_Description

SourceTask      
200131-01-1  0      [[-0.5118116, -0.33358994, 0.265683, 0.2757705...
             1      [[-0.76240206, -0.33975542, 0.09831488, 0.4625...
200133-01-1  2      [[-0.74351245, -0.6697794, 0.44665995, 0.41251...
200151-01-1  3      [[-0.5605606, -0.38250473, 0.14621824, 0.15388...
             4      [[-0.71904707, 0.024312794, 0.016751565, 0.288...
                                          ...                        
ZL-811-02-1  563    [[-0.0940221, -0.39546907, -0.39421532, 0.4003...
             564    [[-0.59371763, -0.70309174, 0.4008928, 0.52645...
             809    [[-0.2944498, 0.009164078, -0.23497285, 0.2615...
ZL-825-01-1  482    [[-0.38767213, -0.15150945, -0.16890076, 0.372...
             810    [[-0.38721478, -0.30592978, -0.37603313, 0.423...
Name: Description, Length: 811, dtype: object

In [10]:
df_iem['description_embedding'] = grouped_df_Description.explode().tolist()
df_iem['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_iem['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [11]:
df_iem[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV21/000097/1021/224,200131-01-1,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,"[-0.5118116, -0.33358994, 0.265683, 0.27577057...","[-0.7624667, -0.5281035, 0.28931808, 0.2974607...","[-0.6633217, -0.45673603, 0.28851366, 0.286182..."
1,HMV21/000097/1021/395,200131-01-1,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,"[-0.76240206, -0.33975542, 0.09831488, 0.46255...","[-0.79862016, -0.5617091, 0.34904265, 0.431891...","[-0.7762478, -0.49182302, 0.27899, 0.43873632,..."
2,HMV21/000097/1021/790,200133-01-1,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,"[-0.74351245, -0.6697794, 0.44665995, 0.412511...","[-0.69442767, -0.6254904, 0.55448556, 0.489238...","[-0.6884283, -0.6241946, 0.5414287, 0.47188964..."
3,HMV21/000097/1021/225,200151-01-1,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,"[-0.5605606, -0.38250473, 0.14621824, 0.153886...","[-0.7726478, -0.4959983, 0.20929237, 0.2909674...","[-0.6885451, -0.4502917, 0.20185933, 0.2440129..."
4,HMV21/000097/1021/362,200151-01-1,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...",CARRIED OUT REPLACEMENT OF CONNECTOR 117MY-A A...,"[-0.71904707, 0.024312794, 0.016751565, 0.2885...","[-0.5845081, -0.44859397, 0.22005159, 0.261085...","[-0.5941808, -0.30838522, 0.1675388, 0.2591248..."


In [12]:
#IGK

In [13]:
grouped_df_Description = df_igk.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_igk.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_igk['description_correction'] = df_igk['Description'] +' '+df_igk['Corrective Action']
grouped_df_description_correction = df_igk.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [14]:
df_igk['description_embedding'] = grouped_df_Description.explode().tolist()
df_igk['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_igk['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [15]:
df_igk[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV20/000008/0520/1,AIRCRAFT ARRIVAL INSPECTION,ON ARRIVAL INSPECTION WING - FUSELAGE & BELLY ...,1. FWD BELLY FAIRING PANEL REMOVAL AND INSTALL...,"[-0.5154262, -0.059293594, -0.20445889, 0.3313...","[-0.48519593, -0.40342197, 0.123832144, 0.3438...","[-0.48169115, -0.33174914, 0.050110325, 0.3213..."
1,HMV20/000008/0520/10,AIRCRAFT ARRIVAL INSPECTION,WHILE CARRYING OUT ARRIVAL CHECK FOUND MULTIPL...,FOR DAMAGES AFT OF AFT LH ENTRY DOOR\nREFER SD...,"[-0.5329796, -0.5712833, 0.22675566, 0.3730128...","[-0.36675638, -0.35657114, 0.028230889, 0.3646...","[-0.48297682, -0.5228271, 0.18049897, 0.375321..."
2,HMV20/000008/0520/100,545116-01-1,"CARRY OUT REMOVAL OF AFT FIXED FAIRING, AFT MO...",1)CARRIED OUT REMOVAL OF LH PYLON AFT FIXED FA...,"[-0.5325942, -0.5919835, 0.20528479, 0.1925752...","[-0.7785467, -0.7075583, 0.6372981, -0.0547787...","[-0.4737005, -0.61133766, 0.34365213, 0.167643..."
3,HMV20/000008/0520/1000,AWR20/001283/0720,WHILE CARRYING OUT INSPECTION OBSERVED MULTIPL...,REFER SDIRC:HMV20/000008/0520/999 FOR DETAILS,"[-0.5325942, -0.5919835, 0.20528479, 0.1925752...","[-0.53626055, -0.5991049, 0.31456485, 0.503281...","[-0.531656, -0.5993713, 0.31005955, 0.4780346,..."
4,HMV20/000008/0520/1001,AWR20/001283/0720,1A. CARRIED OUT BLENDING FOR SCRATCH AND EROSI...,ULTRASONIC THICKNESS INSPECTION CARRIED OUT A...,"[-0.48629203, -0.54483813, 0.36927745, 0.27120...","[-0.6265421, 0.08449781, -0.26386902, 0.367718...","[-0.46380973, -0.5079467, 0.35853922, 0.282353..."


In [16]:
#IKA

In [17]:
grouped_df_Description = df_ika.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_ika.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_ika['description_correction'] = df_ika['Description'] +' '+df_ika['Corrective Action']
grouped_df_description_correction = df_ika.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [18]:
df_ika['description_embedding'] = grouped_df_Description.explode().tolist()
df_ika['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_ika['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [19]:
df_ika[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV21/000050/0821/1,252200-03-1,DURING OPERATIONAL CHECK OF CABIN ATTENDENT SE...,FWD I/B WALL MOUNTED CABIN ATTENDANT SEAT AND ...,"[-0.3395184, -0.30270907, -0.006761565, 0.2737...","[-0.7379921, -0.7228047, 0.41350853, 0.4220378...","[-0.69005334, -0.6494431, 0.3454035, 0.3971541..."
1,HMV21/000050/0821/10,521000-03-1,DURING INSPECTION OBSERVED AFT PASSENGER/CREW ...,AFT RH PASSENGER/CREW DOOR SEAL REPLACED AS PE...,"[-0.13314778, -0.51240563, 0.05247968, 0.07332...","[-0.46523127, -0.45053616, 0.08949266, 0.35440...","[-0.43907294, -0.45560536, 0.08482699, 0.33946..."
2,HMV21/000050/0821/100,AWR21/001909/0821,"AS PER LESSOR FINDING NO. 789, FOUND PAINT DET...","CARRIED OUT REPLACEMENT OF ""SLIDE ARMED"" INDIC...","[-0.52037925, -0.6643813, 0.39566815, 0.450688...","[-0.37792197, -0.5045264, -0.032257516, 0.5318...","[-0.47396562, -0.61535656, 0.28322437, 0.47422..."
3,HMV21/000050/0821/1000,AWR21/004348/0122,FOUND RUB MARKS ON FUSELAGE SKIN B/W FR47-64.F...,1.OBSERVED RUB MARKS ON EXTERNAL FUSELAGE SKIN...,"[-0.5907218, -0.5490377, 0.033942286, 0.346370...","[-0.7514218, -0.4053137, 0.12892392, 0.4112693...","[-0.64391166, -0.4463868, 0.13523637, 0.362702..."
4,HMV21/000050/0821/1001,AWR21/004348/0122,FOUND RUB MARKS ON FUSELAGE SKIN B/W FR64-77.F...,1.OBSERVED RUB MARKS ON EXTERNAL FUSELAGE SKIN...,"[-0.9259125, -0.46554074, -0.2652591, 0.354049...","[-0.90384305, -0.56840605, 0.30424267, 0.49771...","[-0.85098404, -0.5196074, 0.20048687, 0.445723..."


In [20]:
#IKD

In [21]:
grouped_df_Description = df_ikd.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_ikd.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_ikd['description_correction'] = df_ikd['Description'] +' '+df_ikd['Corrective Action']
grouped_df_description_correction = df_ikd.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [22]:
df_ikd['description_embedding'] = grouped_df_Description.explode().tolist()
df_ikd['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_ikd['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [23]:
df_ikd[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV21/000075/0921/1,237100-02-1,"WHILE CARRYING OUT MPD, FOUND CVR RECORDING CO...",TROUBLESHOOTING CARRIED OUT AS PER ASM: 23-71-...,"[-0.69975865, -0.31717533, -0.30507517, 0.4559...","[-0.66564816, -0.572348, 0.21628793, 0.4731656...","[-0.67276955, -0.53930384, 0.1470075, 0.445798..."
1,HMV21/000075/0921/10,AIRCRAFT ARRIVAL INSPECTION,ON ARRIVAL DURING LOOSE EQUIPMENT LIST CHECK O...,"1.PAX LIFE VEST REPLACED AT LOCATIONS ,\n30F,2...","[-0.71008253, -0.062256254, -0.004090868, 0.04...","[-0.7572955, -0.41388062, 0.31189963, 0.353848...","[-0.7237986, -0.345279, 0.2838411, 0.2679734, ..."
2,HMV21/000075/0921/100,ZL-331-01-1,CARRY OUT INSTALLATION OF THS TIP AS PER AMM 5...,REFER IRC NO. HMV21/000075/0921/92 FOR DETAILS...,"[-0.26677307, -0.25749648, -0.45026913, 0.5873...","[-0.5202068, -0.48224446, 0.060794067, 0.51978...","[-0.40793738, -0.4265902, -0.066315904, 0.5437..."
3,HMV21/000075/0921/1000,AWR21/002159/0921,POST RH ENGINE REMOVAL FOUND FOLLOWING DEFECTS...,RH PYLON AFT LOWER FAIRING SPIGOT AND TEFLON C...,"[-0.20877166, -0.2761216, -0.36606097, 0.65601...","[-0.58774304, -0.46796295, -0.02167256, 0.6214...","[-0.4556904, -0.40442207, -0.14013322, 0.63786..."
4,HMV21/000075/0921/1001,AWR21/002332/0921,DURING LESSOR INSP. OBSERVED:\n1. L1 AND R1 DO...,1. L1 AND R1 DOOR SLIDE RAFT COVER: FLOOR SEAL...,"[-0.2315447, -0.44508073, -0.35153654, 0.30105...","[-0.7098748, -0.6646614, 0.4124357, 0.3646254,...","[-0.5661561, -0.60641354, 0.22601867, 0.341724..."


In [24]:
#INU

In [25]:
grouped_df_Description = df_inu.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_inu.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_inu['description_correction'] = df_inu['Description'] +' '+df_inu['Corrective Action']
grouped_df_description_correction = df_inu.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [26]:
df_inu['description_embedding'] = grouped_df_Description.explode().tolist()
df_inu['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_inu['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [27]:
df_inu[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV20/000015/0620/1,AWR20/000807/0620,DURING ARRIVAL INSP OBSERVED FWD CARGO PROTECT...,CARRIED OUT INSTALLATION OF NEW PANEL AS PER A...,NaN,NaN,NaN
1,HMV20/000015/0620/10,AWR20/000807/0620,1. DURING INSPECTION OBSERVED DENT RH SIDE OF ...,REFER SDIRC HMV20/000015/0620/11 FOR REPAIR AC...,NaN,NaN,NaN
2,HMV20/000015/0620/100,AWR20/000807/0620,DURING ARRIVAL INSPECTION OF RH WING AFT MOVAB...,REFER EASA FORM TRACKING NO. E20-S3598 FOR DET...,NaN,NaN,NaN
3,HMV20/000015/0620/1000,AWR20/000974/0720,CARRY OUT HFEC AND MRT AS REQUIRED BY SRM 51-7...,POST DAMAGE REMOVAL HFEC & UT THICKNESS MEASUR...,NaN,NaN,NaN
4,HMV20/000015/0620/1001,AWR20/000975/0720,CARRY OUT HFEC AND MRT OF CHAFING AREA ON BUTT...,POST DAMAGE REMOVAL HFEC & UT THICKNESS MEASUR...,NaN,NaN,NaN


In [28]:
grouped_df_Description = df_ihv.groupby(by='SourceTask')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_ihv.groupby(by='SourceTask')['Corrective Action'].apply(lambda x: x.apply(main))
df_ihv['description_correction'] = df_ihv['Description'] +' '+df_ihv['Corrective Action']
grouped_df_description_correction = df_ihv.groupby(by='SourceTask')['description_correction'].apply(lambda x: x.apply(main))

In [29]:
df_ihv['description_embedding'] = grouped_df_Description.explode().tolist()
df_ihv['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_ihv['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [30]:
df_ihv[['Log Item #', 'SourceTask', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Log Item #,SourceTask,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,HMV21/000138/1221/1,AWR21/004043/0122,"DURING ARRIVAL INSPECTION ,IN REF. PFR 34-52-3...",1. TROUBLESHOOT CARRIED OUT AS PER TSM:34-52-0...,"[-0.49010202, -0.15301369, -0.3246796, 0.36340...","[-0.65616536, -0.48310414, -0.36134672, 0.5439...","[-0.49782938, -0.19084954, -0.34991467, 0.3753..."
1,HMV21/000138/1221/2,AWR21/004043/0122,"DURING ARRIVAL INSPECTION, REF. PFR 23-73-34 N...",1.CARRIED OUT TROUBLESHOOT AS PER TSM: 23-73-0...,"[-0.5803068, -0.28690454, 0.09875969, 0.280003...","[-0.41287166, -0.37747407, 0.07532084, 0.45106...","[-0.4272962, -0.36859867, 0.06173086, 0.431841..."
2,HMV21/000138/1221/3,AWR21/004043/0122,"DURING ARRIVAL INSPECTION,REF. PFR 73-22-34 DM...",1.CARRIED OUT TROUBLESHOOT AS PER TSM:23-73-00...,"[-0.53357804, -0.084838234, -0.41502875, 0.297...","[-0.4173269, -0.2935808, 0.10653887, 0.3169050...","[-0.4230521, -0.2804874, 0.06686468, 0.3186170..."
3,HMV21/000138/1221/4,AWR21/004043/0122,"DURING ARRIVAL INSPECTION, FOUND CREW OXYGEN B...",CARRIED OUT REMOVAL OF CREW OXYGEN CYLINDER AS...,"[-0.6772508, -0.1240623, -0.23572278, 0.265735...","[-0.7018326, -0.3612885, 0.056223277, 0.030126...","[-0.6520622, -0.1935811, -0.19049342, 0.188727..."
4,HMV21/000138/1221/5,AWR21/004043/0122,"DURING ARRIVAL INSPECTION, FOUND 1.LH WING STR...",2.CARRIED OUT REPLACEMENT OF THE LOWER BEACON ...,"[-0.6772508, -0.1240623, -0.23572278, 0.265735...","[-0.394098, -0.42995697, 0.14582756, 0.2566286...","[-0.4179935, -0.39875776, 0.09601926, 0.262992..."


In [31]:
#Similarity matrix for Description

In [32]:
#iem_vec = pd.DataFrame(np.array(df_iem['description_embedding']).reshape(-1, df_iem.shape[0]), columns=df_iem['Log Item #'].tolist()).apply(lambda col: col.explode()).reset_index(drop=True)
#igk_vec = pd.DataFrame(np.array(df_igk['description_embedding']).reshape(-1, df_igk.shape[0]), columns=df_igk['Log Item #'].tolist()).apply(lambda col: col.explode()).reset_index(drop=True)
#ika_vec = pd.DataFrame(np.array(df_ika['description_embedding']).reshape(-1, df_ika.shape[0]), columns=df_ika['Log Item #'].tolist()).apply(lambda col: col.explode()).reset_index(drop=True)
#ikd_vec = pd.DataFrame(np.array(df_ikd['description_embedding']).reshape(-1, df_ikd.shape[0]), columns=df_ikd['Log Item #'].tolist()).apply(lambda col: col.explode()).reset_index(drop=True)
#inu_vec = pd.DataFrame(np.array(df_inu['description_embedding']).reshape(-1, df_inu.shape[0]), columns=df_inu['Log Item #'].tolist()).apply(lambda col: col.explode()).reset_index(drop=True)

In [33]:
#display(iem_vec.head(2))
#display(igk_vec.head(2))
#display(ika_vec.head(2))
#display(ikd_vec.head(2))

In [34]:
#df_description = pd.concat([iem_vec, igk_vec, ika_vec, ikd_vec], axis=1)

In [35]:
#df_description

In [36]:
#cos_sim_description_mat = cosine_similarity(df_description.T)
#cosine_sim_description_df = pd.DataFrame(cos_sim_description_mat, index=df_description.columns, columns=df_description.columns)

In [37]:
#cosine_sim_description_df.to_csv('./cosinematrixfiles/cos_sim_description.csv', index=True)

In [38]:
#TF-IDF Vectorizer

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def main(corpus: list, preserve_symbols=['-', '/']) -> list:
    # Preprocess text for the entire corpus
    preprocessed_corpus = [preprocess_text(text, preserve_symbols) for text in corpus]
    
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    embeddings = vectorizer.fit_transform(preprocessed_corpus)

    # Convert the sparse matrix to dense array
    dense_embeddings = embeddings.toarray()

    return dense_embeddings

In [40]:
#Description

In [41]:
#df_final_description = pd.concat([
    #df_iem[['Log Item #', 'Description']],
    #df_igk[['Log Item #', 'Description']],
    #df_ika[['Log Item #', 'Description']],
    #df_ikd[['Log Item #', 'Description']],
    #df_inu[['Log Item #', 'Description']],
    #df_ihv[['Log Item #', 'Description']]], axis=0).copy()

In [42]:
#desc_tf_idf_vec = main(df_final_description['Description'].tolist(), preserve_symbols=['-', '/'])

In [43]:
#description_embeddings = pd.DataFrame(desc_tf_idf_vec, index=df_final_description['Log Item #'].tolist())
#description_embeddings = description_embeddings.T

In [44]:
#cos_sim_desc_mat = cosine_similarity(description_embeddings.T)
#cosine_sim_desc_df = pd.DataFrame(cos_sim_desc_mat, 
                                 # index=description_embeddings.columns, 
                                 # columns=description_embeddings.columns)

In [45]:
#cosine_sim_desc_df.to_csv('./tfidf/cos_sim_desc.csv', index=True)

In [46]:
#Correction

In [47]:
#df_final_correction = pd.concat([
    #df_iem[['Log Item #', 'Corrective Action']],
    #df_igk[['Log Item #', 'Corrective Action']],
    #df_ika[['Log Item #', 'Corrective Action']],
    #df_ikd[['Log Item #', 'Corrective Action']],
    #df_inu[['Log Item #', 'Corrective Action']]], axis=0).copy()

In [48]:
#correction_tf_idf_vec = main(df_final_correction['Corrective Action'].tolist(), preserve_symbols=['-', '/'])

In [49]:
#correction_embeddings = pd.DataFrame(correction_tf_idf_vec, index=df_final_correction['Log Item #'].tolist())
#correction_embeddings = correction_embeddings.T

In [50]:
#cos_sim_correction_mat = cosine_similarity(correction_embeddings.T)
#cosine_sim_correction_df = pd.DataFrame(cos_sim_correction_mat, 
                                  #index=correction_embeddings.columns, 
                                  #columns=correction_embeddings.columns)

In [51]:
#cosine_sim_correction_df.to_csv('./tfidf/cos_sim_correction.csv', index=True)

In [52]:
# Desc + Correction

In [53]:
df_ikd['description_correction'] = df_ikd['Description'] +' '+df_ikd['Corrective Action']

In [54]:
df_iem['description_correction'] = df_iem['Description'] +' '+df_iem['Corrective Action']

In [55]:
df_igk['description_correction'] = df_igk['Description'] +' '+df_igk['Corrective Action']

In [56]:
df_ika['description_correction'] = df_ika['Description'] +' '+df_ika['Corrective Action']

In [57]:
df_inu['description_correction'] = df_inu['Description'] +' '+df_inu['Corrective Action']

In [58]:
df_ihv['description_correction'] = df_ihv['Description'] +' '+df_ihv['Corrective Action']

In [59]:
df_final_correction_desc = pd.concat([
    df_iem[['Log Item #', 'description_correction']],
    df_igk[['Log Item #', 'description_correction']],
    df_ika[['Log Item #', 'description_correction']],
    df_ikd[['Log Item #', 'description_correction']],
    df_inu[['Log Item #', 'description_correction']],
    df_ihv[['Log Item #', 'description_correction']]], axis=0).copy()

In [60]:
desc_correction_tf_idf_vec = main(df_final_correction_desc['description_correction'].tolist(), preserve_symbols=['-', '/'])

In [61]:
desc_correction_embeddings = pd.DataFrame(desc_correction_tf_idf_vec, index=df_final_correction_desc['Log Item #'].tolist())
desc_correction_embeddings = desc_correction_embeddings.T

In [62]:
cos_sim_desc_correction_mat = cosine_similarity(desc_correction_embeddings.T)
cosine_sim_desc_correction_df = pd.DataFrame(cos_sim_desc_correction_mat, 
                                  index=desc_correction_embeddings.columns, 
                                  columns=desc_correction_embeddings.columns)

In [63]:
#cosine_sim_desc_correction_df.to_csv('./tfidfnew1/cos_sim_desc_correction.csv', index=True)

In [64]:
#TF-IDF Similarity Matrix¶

In [65]:
#Description

In [66]:
#df_des = pd.read_csv('tfidf/cos_sim_desc_correction.csv', index_col=0)

In [67]:
df_des = cosine_sim_desc_correction_df.copy()

In [68]:
df_des

,HMV21/000097/1021/224,HMV21/000097/1021/395,HMV21/000097/1021/790,HMV21/000097/1021/225,HMV21/000097/1021/362,HMV21/000097/1021/364,HMV21/000097/1021/394,HMV21/000097/1021/751,HMV21/000097/1021/422,HMV21/000097/1021/433,...,HMV21/000138/1221/1223,HMV21/000138/1221/1224,HMV21/000138/1221/1225,HMV21/000138/1221/1226,HMV21/000138/1221/1227,HMV21/000138/1221/1228,HMV21/000138/1221/1229,HMV21/000138/1221/1230,HMV21/000138/1221/1231,HMV21/000138/1221/1232
HMV21/000097/1021/224,1.000000,0.276941,0.099270,0.520369,0.230182,0.142236,0.181501,0.094618,0.181922,0.095888,...,0.049972,0.072124,0.117676,0.054894,0.071213,0.055870,0.076004,0.087352,0.079959,0.079265
HMV21/000097/1021/395,0.276941,1.000000,0.212533,0.236933,0.246366,0.093249,0.421938,0.151242,0.113625,0.088211,...,0.057772,0.085189,0.131881,0.069867,0.090515,0.088422,0.046485,0.107919,0.067900,0.083527
HMV21/000097/1021/790,0.099270,0.212533,1.000000,0.097781,0.079277,0.064333,0.104650,0.050288,0.125921,0.086620,...,0.082040,0.188128,0.112882,0.072277,0.244231,0.103673,0.064878,0.138880,0.072716,0.109205
HMV21/000097/1021/225,0.520369,0.236933,0.097781,1.000000,0.239813,0.151814,0.212463,0.139490,0.182442,0.095863,...,0.050745,0.070688,0.116699,0.054337,0.069915,0.054575,0.076354,0.086171,0.078815,0.078465
HMV21/000097/1021/362,0.230182,0.246366,0.079277,0.239813,1.000000,0.120786,0.177834,0.115111,0.089904,0.093844,...,0.037824,0.102586,0.128551,0.061322,0.066311,0.067106,0.045152,0.078810,0.070756,0.062033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HMV21/000138/1221/1228,0.055870,0.088422,0.103673,0.054575,0.067106,0.036205,0.043793,0.033781,0.058171,0.060938,...,0.081053,0.215335,0.149991,0.093600,0.202543,1.000000,0.127623,0.173440,0.225080,0.127567
HMV21/000138/1221/1229,0.076004,0.046485,0.064878,0.076354,0.045152,0.041004,0.024599,0.024592,0.069675,0.046002,...,0.152323,0.126486,0.186466,0.052334,0.127168,0.127623,1.000000,0.133395,0.135072,0.127490
HMV21/000138/1221/1230,0.087352,0.107919,0.138880,0.086171,0.078810,0.043523,0.086224,0.030435,0.082693,0.098771,...,0.079970,0.246749,0.189756,0.048348,0.124662,0.173440,0.133395,1.000000,0.435881,0.398680
HMV21/000138/1221/1231,0.079959,0.067900,0.072716,0.078815,0.070756,0.032440,0.055728,0.054137,0.058186,0.075326,...,0.097976,0.121617,0.165075,0.062451,0.085224,0.225080,0.135072,0.435881,1.000000,0.450909


In [69]:
#df_des = df_des.iloc[:, :5752]
#df_des = df_des.iloc[:5752]
df_des

,HMV21/000097/1021/224,HMV21/000097/1021/395,HMV21/000097/1021/790,HMV21/000097/1021/225,HMV21/000097/1021/362,HMV21/000097/1021/364,HMV21/000097/1021/394,HMV21/000097/1021/751,HMV21/000097/1021/422,HMV21/000097/1021/433,...,HMV21/000138/1221/1223,HMV21/000138/1221/1224,HMV21/000138/1221/1225,HMV21/000138/1221/1226,HMV21/000138/1221/1227,HMV21/000138/1221/1228,HMV21/000138/1221/1229,HMV21/000138/1221/1230,HMV21/000138/1221/1231,HMV21/000138/1221/1232
HMV21/000097/1021/224,1.000000,0.276941,0.099270,0.520369,0.230182,0.142236,0.181501,0.094618,0.181922,0.095888,...,0.049972,0.072124,0.117676,0.054894,0.071213,0.055870,0.076004,0.087352,0.079959,0.079265
HMV21/000097/1021/395,0.276941,1.000000,0.212533,0.236933,0.246366,0.093249,0.421938,0.151242,0.113625,0.088211,...,0.057772,0.085189,0.131881,0.069867,0.090515,0.088422,0.046485,0.107919,0.067900,0.083527
HMV21/000097/1021/790,0.099270,0.212533,1.000000,0.097781,0.079277,0.064333,0.104650,0.050288,0.125921,0.086620,...,0.082040,0.188128,0.112882,0.072277,0.244231,0.103673,0.064878,0.138880,0.072716,0.109205
HMV21/000097/1021/225,0.520369,0.236933,0.097781,1.000000,0.239813,0.151814,0.212463,0.139490,0.182442,0.095863,...,0.050745,0.070688,0.116699,0.054337,0.069915,0.054575,0.076354,0.086171,0.078815,0.078465
HMV21/000097/1021/362,0.230182,0.246366,0.079277,0.239813,1.000000,0.120786,0.177834,0.115111,0.089904,0.093844,...,0.037824,0.102586,0.128551,0.061322,0.066311,0.067106,0.045152,0.078810,0.070756,0.062033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HMV21/000138/1221/1228,0.055870,0.088422,0.103673,0.054575,0.067106,0.036205,0.043793,0.033781,0.058171,0.060938,...,0.081053,0.215335,0.149991,0.093600,0.202543,1.000000,0.127623,0.173440,0.225080,0.127567
HMV21/000138/1221/1229,0.076004,0.046485,0.064878,0.076354,0.045152,0.041004,0.024599,0.024592,0.069675,0.046002,...,0.152323,0.126486,0.186466,0.052334,0.127168,0.127623,1.000000,0.133395,0.135072,0.127490
HMV21/000138/1221/1230,0.087352,0.107919,0.138880,0.086171,0.078810,0.043523,0.086224,0.030435,0.082693,0.098771,...,0.079970,0.246749,0.189756,0.048348,0.124662,0.173440,0.133395,1.000000,0.435881,0.398680
HMV21/000138/1221/1231,0.079959,0.067900,0.072716,0.078815,0.070756,0.032440,0.055728,0.054137,0.058186,0.075326,...,0.097976,0.121617,0.165075,0.062451,0.085224,0.225080,0.135072,0.435881,1.000000,0.450909


In [70]:
#cosine_sim_desc_correction_df = df_des.copy()
cosine_sim_desc_correction_df 

,HMV21/000097/1021/224,HMV21/000097/1021/395,HMV21/000097/1021/790,HMV21/000097/1021/225,HMV21/000097/1021/362,HMV21/000097/1021/364,HMV21/000097/1021/394,HMV21/000097/1021/751,HMV21/000097/1021/422,HMV21/000097/1021/433,...,HMV21/000138/1221/1223,HMV21/000138/1221/1224,HMV21/000138/1221/1225,HMV21/000138/1221/1226,HMV21/000138/1221/1227,HMV21/000138/1221/1228,HMV21/000138/1221/1229,HMV21/000138/1221/1230,HMV21/000138/1221/1231,HMV21/000138/1221/1232
HMV21/000097/1021/224,1.000000,0.276941,0.099270,0.520369,0.230182,0.142236,0.181501,0.094618,0.181922,0.095888,...,0.049972,0.072124,0.117676,0.054894,0.071213,0.055870,0.076004,0.087352,0.079959,0.079265
HMV21/000097/1021/395,0.276941,1.000000,0.212533,0.236933,0.246366,0.093249,0.421938,0.151242,0.113625,0.088211,...,0.057772,0.085189,0.131881,0.069867,0.090515,0.088422,0.046485,0.107919,0.067900,0.083527
HMV21/000097/1021/790,0.099270,0.212533,1.000000,0.097781,0.079277,0.064333,0.104650,0.050288,0.125921,0.086620,...,0.082040,0.188128,0.112882,0.072277,0.244231,0.103673,0.064878,0.138880,0.072716,0.109205
HMV21/000097/1021/225,0.520369,0.236933,0.097781,1.000000,0.239813,0.151814,0.212463,0.139490,0.182442,0.095863,...,0.050745,0.070688,0.116699,0.054337,0.069915,0.054575,0.076354,0.086171,0.078815,0.078465
HMV21/000097/1021/362,0.230182,0.246366,0.079277,0.239813,1.000000,0.120786,0.177834,0.115111,0.089904,0.093844,...,0.037824,0.102586,0.128551,0.061322,0.066311,0.067106,0.045152,0.078810,0.070756,0.062033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HMV21/000138/1221/1228,0.055870,0.088422,0.103673,0.054575,0.067106,0.036205,0.043793,0.033781,0.058171,0.060938,...,0.081053,0.215335,0.149991,0.093600,0.202543,1.000000,0.127623,0.173440,0.225080,0.127567
HMV21/000138/1221/1229,0.076004,0.046485,0.064878,0.076354,0.045152,0.041004,0.024599,0.024592,0.069675,0.046002,...,0.152323,0.126486,0.186466,0.052334,0.127168,0.127623,1.000000,0.133395,0.135072,0.127490
HMV21/000138/1221/1230,0.087352,0.107919,0.138880,0.086171,0.078810,0.043523,0.086224,0.030435,0.082693,0.098771,...,0.079970,0.246749,0.189756,0.048348,0.124662,0.173440,0.133395,1.000000,0.435881,0.398680
HMV21/000138/1221/1231,0.079959,0.067900,0.072716,0.078815,0.070756,0.032440,0.055728,0.054137,0.058186,0.075326,...,0.097976,0.121617,0.165075,0.062451,0.085224,0.225080,0.135072,0.435881,1.000000,0.450909


In [71]:
#df_des.iloc[:, 0].to_csv('./tfidfnew/first_column_df_des.csv', index=True) 

In [72]:
#df_des.iloc[[0]].to_csv('./tfidfnew/first_row_df_des.csv', index=True) 

In [73]:
#df_des.head()

In [74]:
#upper_triangular_values = get_upper_triangular_values(df_des.values)

In [75]:
#upper_triangular_values = np.array(upper_triangular_values)

In [76]:
#np.median(upper_triangular_values), np.mean(upper_triangular_values), np.std(upper_triangular_values)

In [77]:
#import seaborn as sns

In [78]:
#import seaborn as sns

In [79]:
#sns.kdeplot(upper_triangular_values)

In [80]:
#for i in np.arange(98, 100, 0.5): 
    #print(i, np.percentile(upper_triangular_values, i))

In [81]:
df_des= threshold_transform(df_des, threshold=0.5)
df_des

,HMV21/000097/1021/224,HMV21/000097/1021/395,HMV21/000097/1021/790,HMV21/000097/1021/225,HMV21/000097/1021/362,HMV21/000097/1021/364,HMV21/000097/1021/394,HMV21/000097/1021/751,HMV21/000097/1021/422,HMV21/000097/1021/433,...,HMV21/000138/1221/1223,HMV21/000138/1221/1224,HMV21/000138/1221/1225,HMV21/000138/1221/1226,HMV21/000138/1221/1227,HMV21/000138/1221/1228,HMV21/000138/1221/1229,HMV21/000138/1221/1230,HMV21/000138/1221/1231,HMV21/000138/1221/1232
HMV21/000097/1021/224,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/395,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/790,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/225,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/362,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HMV21/000138/1221/1228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HMV21/000138/1221/1229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
HMV21/000138/1221/1230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
HMV21/000138/1221/1231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [82]:
#task = pd.concat([
    #df_iem[['Log Item #', 'SourceTask', 'Description']],
    #df_igk[['Log Item #', 'SourceTask', 'Description']],
    #df_ika[['Log Item #', 'SourceTask', 'Description']],
    #df_ikd[['Log Item #', 'SourceTask', 'Description']],
    #df_inu[['Log Item #', 'SourceTask', 'Description']]], axis=0).copy()

In [83]:
#task

In [84]:
#df_des.to_csv('./tfidf/Desc_ones_zeros.csv', index=True)

In [85]:
#obsv_cluster = dict()

#for i in range(df_des.shape[0]):
    #for j in range(i+1, df_des.shape[1]):
        #if df_des.iloc[i, j] == 1:
           # if df_des.columns[i] != df_des.columns[j]:
              #  if df_des.columns[i] in obsv_cluster:
                   # obsv_cluster[df_des.columns[i]].append(df_des.columns[j])
                #else:
                   # obsv_cluster[df_des.columns[i]] = [df_des.columns[j]]

In [86]:
#obsv_cluster

In [87]:
#task_obsv_cluster = dict()
#for i in list(obsv_cluster.items()):
    #if str(task[task['Log Item #'] == i[0]]['SourceTask'].values.tolist()[0]) in task_obsv_cluster:
        #task_obsv_cluster[str(task[task['Log Item #'] == i[0]]['SourceTask'].values.tolist()[0])].append({i[0]:i[1]})
    #else:
       # task_obsv_cluster[str(task[task['Log Item #'] == i[0]]['SourceTask'].values.tolist()[0])] = [{i[0]:i[1]}]   

In [88]:
#task_obsv_cluster

In [89]:
#obsv_cluster

In [90]:
#from pyvis.network import Network

# Create a pyvis network instance
#net = Network(height="800px", width="100%", notebook=True)

# Add nodes to the network
#for key, values in obsv_cluster.items():
  #  net.add_node(key)
  #  for value in values:
   #     net.add_node(value)  
   #     net.add_edge(key, value)



In [91]:
# Get clusters based on edge relationships
#clusters = {}
#for edge in net.edges:
#    source = edge['from']
#    target = edge['to']

#    if source not in clusters:
#        clusters[source] = set([source, target])
#    else:
 #       clusters[source].add(target)
#
# Convert clusters to a dictionary where values are lists
#clusters_list = {key: list(value) for key, value in clusters.items()}

# Print clusters as lists
#cnt = 0
#for key, value in clusters_list.items():
#    cnt += 1
#    print(f"Group : {value}")

#print(f"Total groups : {cnt}")

In [92]:
#list(clusters.values())

In [93]:
#def create_groups(edges):
    # Create an adjacency list from the edges
#    adj_list = {}
#    for edge in edges:
#        for node in edge:
#            adj_list.setdefault(node, set()).update(edge - {node})

    # Function to perform depth-first search (DFS)
#    def dfs(node, visited, group):
#        visited.add(node)
#        group.add(node)
#        for neighbor in adj_list.get(node, []):
#            if neighbor not in visited:
#                dfs(neighbor, visited, group)
    # Initialize variables
#    visited = set()
#    groups = []

    # Traverse the graph using DFS
#    for node in adj_list.keys():
#        if node not in visited:
#            group = set()
#            dfs(node, visited, group)
#            groups.append(list(group))

#   return groups


#source_task_logitems = create_groups(list(clusters.values()))
#print(source_task_logitems)


In [94]:
#source_task_logitems

In [95]:
#for i in source_task_logitems:
#    print(len(i))

In [96]:
#task

In [97]:
#source_task_logitems
# type(source_task_logitems)

In [98]:
#src_task_groups = []
#for i in source_task_logitems:
#    temp = []
#    for j in i:
#        temp.append(task[task['Log Item #'] == j]['SourceTask'].values[0])
#    src_task_groups.append(list(set(temp)))  # remove duplicates in temp)
#src_task_groups

In [99]:
#task['SourceTask'].value_counts()

In [100]:
#src_task_description = []
#for i in source_task_logitems:
#    temp = []
#    for j in i:
#        temp.append(task[task['Log Item #'] == j]['Description'].values[0])
#    src_task_description.append(temp)
#src_task_description

In [101]:
#result_dict = {}

# Zip the corresponding elements from three lists
#zipped_elements = zip(source_task_logitems, src_task_groups, src_task_description)

# Iterate over the zipped elements to create the result dictionary
#for i, elements in enumerate(zipped_elements, start=1):
   # group_key = f'group_{i}'
   # result_dict[group_key] = {
    #    'logItem': elements[0],
    #    'sourceTask': elements[1],
    #    'description': elements[2],
     #   'frequency': len(elements[0]),
  #  }

# Print the resulting dictionary
#print(result_dict)

In [102]:
#result_dict.keys()

In [103]:
#result_dict = {}

# Zip the corresponding elements from three lists
#zipped_elements = zip(source_task_logitems, src_task_groups, src_task_description)

# Iterate over the zipped elements to create the result dictionary
#for i, elements in enumerate(zipped_elements, start=1):
#    group_key = f'group_{i}'
#    result_dict[group_key] = {
#        'logItem': elements[0],
#        'sourceTask': elements[1],
#        'description': elements[2],
#        'frequency': len(elements[0]),
#    }

# Print the resulting dictionary
#print(result_dict)

In [104]:
#result_dict['group_1']

In [105]:
#%system jupyter notebook --ServerApp.iopub_data_rate_limit=10000000

In [106]:
#final_result = pd.DataFrame(result_dict.values())
#final_result.head()

In [107]:
#final_result.shape

In [108]:
#type(final_result)

In [109]:
#final_result = final_result[['sourceTask', 'logItem', 'description', 'frequency']]
#final_result.head()

In [110]:
#final_result.to_excel('./tfidf/maintenance_description_result.xlsx', index=False)

In [111]:
df_des

,HMV21/000097/1021/224,HMV21/000097/1021/395,HMV21/000097/1021/790,HMV21/000097/1021/225,HMV21/000097/1021/362,HMV21/000097/1021/364,HMV21/000097/1021/394,HMV21/000097/1021/751,HMV21/000097/1021/422,HMV21/000097/1021/433,...,HMV21/000138/1221/1223,HMV21/000138/1221/1224,HMV21/000138/1221/1225,HMV21/000138/1221/1226,HMV21/000138/1221/1227,HMV21/000138/1221/1228,HMV21/000138/1221/1229,HMV21/000138/1221/1230,HMV21/000138/1221/1231,HMV21/000138/1221/1232
HMV21/000097/1021/224,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/395,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/790,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/225,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HMV21/000097/1021/362,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HMV21/000138/1221/1228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HMV21/000138/1221/1229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
HMV21/000138/1221/1230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
HMV21/000138/1221/1231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [112]:
import pandas as pd

# Assuming df_des is your dataframe without column or row names

# Reset index to convert it into a regular column
df_des_reset = df_des.reset_index()

# Unpivot the dataframe
df_unpivoted = df_des_reset.melt(id_vars='index', var_name='obsid_d', value_name='Value')

# Set the column names
df_unpivoted.columns = ['obsid_s', 'obsid_d', 'Value']

# Set the index back
df_unpivoted.set_index('obsid_s', inplace=True)

# Display the unpivoted dataframe
df_unpivoted
 

,obsid_d,Value
obsid_s,,
HMV21/000097/1021/224,HMV21/000097/1021/224,1
HMV21/000097/1021/395,HMV21/000097/1021/224,0
HMV21/000097/1021/790,HMV21/000097/1021/224,0
HMV21/000097/1021/225,HMV21/000097/1021/224,1
HMV21/000097/1021/362,HMV21/000097/1021/224,0
...,...,...
HMV21/000138/1221/1228,HMV21/000138/1221/1232,0
HMV21/000138/1221/1229,HMV21/000138/1221/1232,0
HMV21/000138/1221/1230,HMV21/000138/1221/1232,0


In [113]:
#df_final_description

In [114]:
#df_unpivoted.to_csv('./tfidf/df_unpivoted.csv', index=True)

In [115]:
#df_iem

In [116]:
#column_headers = df_iem.columns

# Display the column headers
#column_headers

In [117]:
# Assuming df_iem is your DataFrame
#df_iem_selected = df_iem[['Log Item #', 'SourceTask']]
#df_iem_selected

In [118]:
# Assuming df_unpivoted is your DataFrame
df_unpivot = df_unpivoted[df_unpivoted['obsid_d'] != 'level_0']

# Printing the DataFrame after filtering
df_unpivot


,obsid_d,Value
obsid_s,,
HMV21/000097/1021/224,HMV21/000097/1021/224,1
HMV21/000097/1021/395,HMV21/000097/1021/224,0
HMV21/000097/1021/790,HMV21/000097/1021/224,0
HMV21/000097/1021/225,HMV21/000097/1021/224,1
HMV21/000097/1021/362,HMV21/000097/1021/224,0
...,...,...
HMV21/000138/1221/1228,HMV21/000138/1221/1232,0
HMV21/000138/1221/1229,HMV21/000138/1221/1232,0
HMV21/000138/1221/1230,HMV21/000138/1221/1232,0


In [119]:
# Assuming df1, df2, df3, df4, and df5 are your DataFrames
# Concatenate the DataFrames
combined_df = pd.concat([df_iem, df_igk, df_ika, df_ikd, df_inu, df_ihv], ignore_index=True)

# Printing the combined DataFrame
combined_df


,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center,description_correction,description_embedding,corrective_action_embedding,description_correction_embedding,SourceTask1
0,No,MIREP,HMV21/000097/1021/224,24.0,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/224,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...","[-0.5118116, -0.33358994, 0.265683, 0.27577057...","[-0.7624667, -0.5281035, 0.28931808, 0.2974607...","[-0.6633217, -0.45673603, 0.28851366, 0.286182...",NaN
1,No,MIREP,HMV21/000097/1021/395,24.0,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,HMV21/000097/1021/395,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...","[-0.76240206, -0.33975542, 0.09831488, 0.46255...","[-0.79862016, -0.5617091, 0.34904265, 0.431891...","[-0.7762478, -0.49182302, 0.27899, 0.43873632,...",NaN
2,No,MIREP,HMV21/000097/1021/790,52.0,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,HMV21/000097/1021/790,Close,Closed,200133-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...","[-0.74351245, -0.6697794, 0.44665995, 0.412511...","[-0.69442767, -0.6254904, 0.55448556, 0.489238...","[-0.6884283, -0.6241946, 0.5414287, 0.47188964...",NaN
3,No,MIREP,HMV21/000097/1021/225,24.0,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/225,Close,Closed,200151-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...","[-0.5605606, -0.38250473, 0.14621824, 0.153886...","[-0.7726478, -0.4959983, 0.20929237, 0.2909674...","[-0.6885451, -0.4502917, 0.20185933, 0.2440129...",NaN
4,No,MIREP,HMV21/000097/1021/362,24.0,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...",CARRIED OUT REPLACEMENT OF CONNECTOR 117MY-A A...,HMV21/000097/1021/362,Close,Closed,200151-01-1,...,NaN,NaN,NaN,NaN,NaN,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...","[-0.71904707, 0.024312794, 0.016751565, 0.2885...","[-0.5845081, -0.44859397, 0.22005159, 0.261085...","[-0.5941808, -0.30838522, 0.1675388, 0.2591248...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,No,MIREP,HMV21/000138/1221/1228,31.0,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,CARRIED OUT TROUBLESHOOTING OF AIDS DATA NOT S...,HMV21/000138/1221/1228,Close,Closed,AWR21/005357/0322,...,NaN,NaN,NaN,NaN,NaN,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,"[-0.6396081, -0.2312267, 0.034146167, -0.11750...","[-0.71913266, -0.70069474, 0.58397484, 0.39729...","[-0.70333195, -0.56126887, 0.4623483, 0.261023...",NaN
7019,No,MIREP,HMV21/000138/1221/1229,28.0,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...",TROUBLESHOOTING FOR THE FAULT CARRIED OUT AS P...,HMV21/000138/1221/1229,Close,Closed,AWR22/004644/1122,...,NaN,NaN,NaN,NaN,NaN,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...","[-0.35522807, -0.3229499, 0.024949888, 0.21241...","[-0.6559304, -0.55850506, 0.5316232, 0.5625286...","[-0.59365004, -0.5045403, 0.42566532, 0.497925...",NaN
7020,No,MIREP,HMV21/000138/1221/1230,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AND TESTS AS PER S...,HMV21/000138/1221/1230,Close,Closed,AWR22/004750/1122,...,NaN,NaN,NaN,NaN,NaN,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,"[-0.58108073, -0.36657876, 0.010545301, 0.5320...","[-0.5646659, -0.73799634, 0.5119383, 0.5037746...","[-0.565673, -0.67673814, 0.45982942, 0.4918415...",NaN
7021,No,MIREP,HMV21/000138/1221/1231,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AS PER TSM 31-32-0...,HMV21/000138/1221/1231,Close,Closed,AWR22/0047

In [120]:
#combined_df = combined_df.iloc[:5752]
combined_df

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center,description_correction,description_embedding,corrective_action_embedding,description_correction_embedding,SourceTask1
0,No,MIREP,HMV21/000097/1021/224,24.0,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/224,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...","[-0.5118116, -0.33358994, 0.265683, 0.27577057...","[-0.7624667, -0.5281035, 0.28931808, 0.2974607...","[-0.6633217, -0.45673603, 0.28851366, 0.286182...",NaN
1,No,MIREP,HMV21/000097/1021/395,24.0,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,HMV21/000097/1021/395,Close,Closed,200131-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...","[-0.76240206, -0.33975542, 0.09831488, 0.46255...","[-0.79862016, -0.5617091, 0.34904265, 0.431891...","[-0.7762478, -0.49182302, 0.27899, 0.43873632,...",NaN
2,No,MIREP,HMV21/000097/1021/790,52.0,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,HMV21/000097/1021/790,Close,Closed,200133-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...","[-0.74351245, -0.6697794, 0.44665995, 0.412511...","[-0.69442767, -0.6254904, 0.55448556, 0.489238...","[-0.6884283, -0.6241946, 0.5414287, 0.47188964...",NaN
3,No,MIREP,HMV21/000097/1021/225,24.0,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/225,Close,Closed,200151-01-1,...,NaN,NaN,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...","[-0.5605606, -0.38250473, 0.14621824, 0.153886...","[-0.7726478, -0.4959983, 0.20929237, 0.2909674...","[-0.6885451, -0.4502917, 0.20185933, 0.2440129...",NaN
4,No,MIREP,HMV21/000097/1021/362,24.0,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...",CARRIED OUT REPLACEMENT OF CONNECTOR 117MY-A A...,HMV21/000097/1021/362,Close,Closed,200151-01-1,...,NaN,NaN,NaN,NaN,NaN,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...","[-0.71904707, 0.024312794, 0.016751565, 0.2885...","[-0.5845081, -0.44859397, 0.22005159, 0.261085...","[-0.5941808, -0.30838522, 0.1675388, 0.2591248...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,No,MIREP,HMV21/000138/1221/1228,31.0,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,CARRIED OUT TROUBLESHOOTING OF AIDS DATA NOT S...,HMV21/000138/1221/1228,Close,Closed,AWR21/005357/0322,...,NaN,NaN,NaN,NaN,NaN,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,"[-0.6396081, -0.2312267, 0.034146167, -0.11750...","[-0.71913266, -0.70069474, 0.58397484, 0.39729...","[-0.70333195, -0.56126887, 0.4623483, 0.261023...",NaN
7019,No,MIREP,HMV21/000138/1221/1229,28.0,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...",TROUBLESHOOTING FOR THE FAULT CARRIED OUT AS P...,HMV21/000138/1221/1229,Close,Closed,AWR22/004644/1122,...,NaN,NaN,NaN,NaN,NaN,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...","[-0.35522807, -0.3229499, 0.024949888, 0.21241...","[-0.6559304, -0.55850506, 0.5316232, 0.5625286...","[-0.59365004, -0.5045403, 0.42566532, 0.497925...",NaN
7020,No,MIREP,HMV21/000138/1221/1230,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AND TESTS AS PER S...,HMV21/000138/1221/1230,Close,Closed,AWR22/004750/1122,...,NaN,NaN,NaN,NaN,NaN,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,"[-0.58108073, -0.36657876, 0.010545301, 0.5320...","[-0.5646659, -0.73799634, 0.5119383, 0.5037746...","[-0.565673, -0.67673814, 0.45982942, 0.4918415...",NaN
7021,No,MIREP,HMV21/000138/1221/1231,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AS PER TSM 31-32-0...,HMV21/000138/1221/1231,Close,Closed,AWR22/0047

In [121]:


# Assuming df_unpivot and combined_df are your DataFrames

# Reset the index of df_unpivot so that 'obsid_s' becomes a column
df_unpivot.reset_index(inplace=True)

# Merge df_unpivot with combined_df on 'Log Item #' to get 'sourcetask_s'
df_unpivot = pd.merge(df_unpivot, combined_df[['Log Item #', 'SourceTask']], left_on='obsid_s', right_on='Log Item #', how='left')
df_unpivot.rename(columns={'SourceTask': 'sourcetask_s'}, inplace=True)
df_unpivot.drop(columns='Log Item #', inplace=True)

# Merge df_unpivot with combined_df again on 'Log Item #' to get 'sourcetask_d'
df_unpivot = pd.merge(df_unpivot, combined_df[['Log Item #', 'SourceTask']], left_on='obsid_d', right_on='Log Item #', how='left')
df_unpivot.rename(columns={'SourceTask': 'sourcetask_d'}, inplace=True)
df_unpivot.drop(columns='Log Item #', inplace=True)

# Printing the updated DataFrame
df_unpivot



,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
0,HMV21/000097/1021/224,HMV21/000097/1021/224,1,200131-01-1,200131-01-1
1,HMV21/000097/1021/395,HMV21/000097/1021/224,0,200131-01-1,200131-01-1
2,HMV21/000097/1021/790,HMV21/000097/1021/224,0,200133-01-1,200131-01-1
3,HMV21/000097/1021/225,HMV21/000097/1021/224,1,200151-01-1,200131-01-1
4,HMV21/000097/1021/362,HMV21/000097/1021/224,0,200151-01-1,200131-01-1
...,...,...,...,...,...
72335020,HMV21/000138/1221/1228,HMV21/000138/1221/1232,0,AWR21/005357/0322,AWR22/004750/1122
72335021,HMV21/000138/1221/1229,HMV21/000138/1221/1232,0,AWR22/004644/1122,AWR22/004750/1122
72335022,HMV21/000138/1221/1230,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122
72335023,HMV21/000138/1221/1231,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122


In [122]:
#combined_df.to_csv('./tfidf/combined_df.csv', index=True)

In [123]:
df_unpivot

,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
0,HMV21/000097/1021/224,HMV21/000097/1021/224,1,200131-01-1,200131-01-1
1,HMV21/000097/1021/395,HMV21/000097/1021/224,0,200131-01-1,200131-01-1
2,HMV21/000097/1021/790,HMV21/000097/1021/224,0,200133-01-1,200131-01-1
3,HMV21/000097/1021/225,HMV21/000097/1021/224,1,200151-01-1,200131-01-1
4,HMV21/000097/1021/362,HMV21/000097/1021/224,0,200151-01-1,200131-01-1
...,...,...,...,...,...
72335020,HMV21/000138/1221/1228,HMV21/000138/1221/1232,0,AWR21/005357/0322,AWR22/004750/1122
72335021,HMV21/000138/1221/1229,HMV21/000138/1221/1232,0,AWR22/004644/1122,AWR22/004750/1122
72335022,HMV21/000138/1221/1230,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122
72335023,HMV21/000138/1221/1231,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122


In [124]:
#df_unpivot.to_csv('./tfidf/unpivot.csv', index=True)

In [125]:
df_unpivot

,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
0,HMV21/000097/1021/224,HMV21/000097/1021/224,1,200131-01-1,200131-01-1
1,HMV21/000097/1021/395,HMV21/000097/1021/224,0,200131-01-1,200131-01-1
2,HMV21/000097/1021/790,HMV21/000097/1021/224,0,200133-01-1,200131-01-1
3,HMV21/000097/1021/225,HMV21/000097/1021/224,1,200151-01-1,200131-01-1
4,HMV21/000097/1021/362,HMV21/000097/1021/224,0,200151-01-1,200131-01-1
...,...,...,...,...,...
72335020,HMV21/000138/1221/1228,HMV21/000138/1221/1232,0,AWR21/005357/0322,AWR22/004750/1122
72335021,HMV21/000138/1221/1229,HMV21/000138/1221/1232,0,AWR22/004644/1122,AWR22/004750/1122
72335022,HMV21/000138/1221/1230,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122
72335023,HMV21/000138/1221/1231,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122


In [126]:
# Assuming df_unpivot is your original DataFrame
# Splitting the DataFrame into three chunks
chunk_size = len(df_unpivot) // 3
chunks = [df_unpivot[i:i+chunk_size] for i in range(0, len(df_unpivot), chunk_size)]

# Define the custom function to update the 'Value' column based on conditions
def update_value(row):
    if row['Value'] == 0:
        return 0
    elif row['sourcetask_s'] == row['sourcetask_d']:
        return 1
    else:
        return 0

# Process each chunk individually
processed_chunks = []
for chunk in chunks:
    # Create a copy of the chunk to work with
    chunk_copy = chunk.copy()
    # Apply the custom function row-wise to update the 'Value' column
    chunk_copy['Value'] = chunk_copy.apply(update_value, axis=1)
    # Append the processed chunk to the list
    processed_chunks.append(chunk_copy)

# Concatenate the processed chunks into a single DataFrame
df_unpivotchk = pd.concat(processed_chunks)

# Optionally, you can save the concatenated DataFrame to a CSV file
# df_result.to_csv('./tfidf/df_result.csv', index=False)

# Optionally, you can print the concatenated DataFrame
df_unpivotchk


,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
0,HMV21/000097/1021/224,HMV21/000097/1021/224,1,200131-01-1,200131-01-1
1,HMV21/000097/1021/395,HMV21/000097/1021/224,0,200131-01-1,200131-01-1
2,HMV21/000097/1021/790,HMV21/000097/1021/224,0,200133-01-1,200131-01-1
3,HMV21/000097/1021/225,HMV21/000097/1021/224,0,200151-01-1,200131-01-1
4,HMV21/000097/1021/362,HMV21/000097/1021/224,0,200151-01-1,200131-01-1
...,...,...,...,...,...
72335020,HMV21/000138/1221/1228,HMV21/000138/1221/1232,0,AWR21/005357/0322,AWR22/004750/1122
72335021,HMV21/000138/1221/1229,HMV21/000138/1221/1232,0,AWR22/004644/1122,AWR22/004750/1122
72335022,HMV21/000138/1221/1230,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122
72335023,HMV21/000138/1221/1231,HMV21/000138/1221/1232,0,AWR22/004750/1122,AWR22/004750/1122


In [127]:
# Assuming df_unpivot is your DataFrame

# Create a copy of df_unpivot to work with
#df_unpivotchk = df_unpivot.copy()

# Define a custom function to update the 'value' column based on conditions
#def update_value(row):
 #   if row['Value'] == 0:
  #      return 0
   # elif row['sourcetask_s'] == row['sourcetask_d']:
    #    return 1
    #else:
     #   return 0

# Apply the custom function row-wise to update the 'value' column
#df_unpivotchk['Value'] = df_unpivotchk.apply(update_value, axis=1)

# Saving the transformed DataFrame as df_unpivotchk
#df_unpivotchk.to_csv('./tfidf/df_unpivotchk.csv', index=False)  # Or use any other desired file format

# Optionally, you can also print the DataFrame if needed
#df_unpivotchk


In [128]:
#df_unpivotchk = df_result.copy()

In [129]:
# Assuming df_unpivotchk is your DataFrame

# Filter out rows where 'value' is 1
df_sim = df_unpivotchk[df_unpivotchk['Value'] == 1].copy()

# Reset the index of df_sim if needed
# df_sim.reset_index(drop=True, inplace=True)

# Printing the resulting DataFrame
df_sim


,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
0,HMV21/000097/1021/224,HMV21/000097/1021/224,1,200131-01-1,200131-01-1
8506,HMV21/000097/1021/395,HMV21/000097/1021/395,1,200131-01-1,200131-01-1
12817,HMV21/000075/0921/721,HMV21/000097/1021/395,1,200131-01-1,200131-01-1
17012,HMV21/000097/1021/790,HMV21/000097/1021/790,1,200133-01-1,200133-01-1
25518,HMV21/000097/1021/225,HMV21/000097/1021/225,1,200151-01-1,200151-01-1
...,...,...,...,...,...
72301000,HMV21/000138/1221/1228,HMV21/000138/1221/1228,1,AWR21/005357/0322,AWR21/005357/0322
72309506,HMV21/000138/1221/1229,HMV21/000138/1221/1229,1,AWR22/004644/1122,AWR22/004644/1122
72318012,HMV21/000138/1221/1230,HMV21/000138/1221/1230,1,AWR22/004750/1122,AWR22/004750/1122
72326518,HMV21/000138/1221/1231,HMV21/000138/1221/1231,1,AWR22/004750/1122,AWR22/004750/1122


In [130]:
#df_sim.to_csv('./tfidf/df_similar.csv', index=True)

In [131]:
# Assuming df_sim is your DataFrame

# Remove rows where 'obsid_s' is equal to 'obsid_d'
df_sim1 = df_sim[df_sim['obsid_s'] != df_sim['obsid_d']].copy()

# Reset the index of df_sim1 if needed
# df_sim1.reset_index(drop=True, inplace=True)

# Optionally, you can also print the DataFrame if needed
df_sim1


,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d
12817,HMV21/000075/0921/721,HMV21/000097/1021/395,1,200131-01-1,200131-01-1
36504,HMV21/000050/0821/441,HMV21/000097/1021/362,1,200151-01-1,200151-01-1
43043,HMV21/000097/1021/500,HMV21/000097/1021/364,1,200151-01-1,200151-01-1
85439,HMV21/000097/1021/506,HMV21/000097/1021/154,1,200453-01-1,200453-01-1
119085,HMV21/000097/1021/44,HMV21/000097/1021/43,1,215222-01-1,215222-01-1
...,...,...,...,...,...
72207435,HMV21/000138/1221/1218,HMV21/000138/1221/1217,1,ZL-500-02-1,ZL-500-02-1
72215936,HMV21/000138/1221/1214,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1
72215937,HMV21/000138/1221/1215,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1
72215938,HMV21/000138/1221/1216,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1


In [132]:
import networkx as nx

# Assuming df_sim1 is your DataFrame

# Create a directed graph
G = nx.DiGraph()

# Add edges based on connections between obsid_s and obsid_d
for index, row in df_sim1.iterrows():
    G.add_edge(row['obsid_s'], row['obsid_d'])

# Assign groups using strongly connected components
groups = {node: i for i, component in enumerate(nx.strongly_connected_components(G), start=1) for node in component}

# Map the groups to the DataFrame
df_sim1['Group'] = df_sim1['obsid_s'].map(groups)

# Printing the resulting DataFrame
df_sim1


,obsid_s,obsid_d,Value,sourcetask_s,sourcetask_d,Group
12817,HMV21/000075/0921/721,HMV21/000097/1021/395,1,200131-01-1,200131-01-1,1
36504,HMV21/000050/0821/441,HMV21/000097/1021/362,1,200151-01-1,200151-01-1,2
43043,HMV21/000097/1021/500,HMV21/000097/1021/364,1,200151-01-1,200151-01-1,3
85439,HMV21/000097/1021/506,HMV21/000097/1021/154,1,200453-01-1,200453-01-1,4
119085,HMV21/000097/1021/44,HMV21/000097/1021/43,1,215222-01-1,215222-01-1,5
...,...,...,...,...,...,...
72207435,HMV21/000138/1221/1218,HMV21/000138/1221/1217,1,ZL-500-02-1,ZL-500-02-1,1105
72215936,HMV21/000138/1221/1214,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1,1105
72215937,HMV21/000138/1221/1215,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1,1105
72215938,HMV21/000138/1221/1216,HMV21/000138/1221/1218,1,ZL-500-02-1,ZL-500-02-1,1105


In [133]:
#df_sim1.to_csv('./tfidf5/df_group.csv', index=True)

In [134]:
# Assuming df_sim1 is your DataFrame

# Remove duplicate items in column 'obsid_s'
#df_group = df_sim1.drop_duplicates(subset=['obsid_s']).copy()

# Reset the index of df_group if needed
# df_group.reset_index(drop=True, inplace=True)

# Saving the transformed DataFrame as df_group
#df_group.to_csv('./tfidf1/df_group.csv', index=True)

# Optionally, you can also print the DataFrame if needed
#df_group


In [135]:
# Assuming combined_df and df_group are your DataFrames

# Merge combined_df with df_group on 'Log Item #' and 'obsid_s' to get 'group' values
group_df = pd.merge(combined_df, df_sim1[['obsid_s', 'Group']], left_on='Log Item #', right_on='obsid_s', how='left')

# Rename the 'Group' column to 'group'
group_df.rename(columns={'Group': 'group'}, inplace=True)

# Fill NaN values with 0 in the 'group' column
group_df['group'].fillna(0, inplace=True)

# Optionally, you can print the resulting DataFrame if needed
group_df


,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Reason for Deferral,Auth. Ref. #,Message Center,description_correction,description_embedding,corrective_action_embedding,description_correction_embedding,SourceTask1,obsid_s,group
0,No,MIREP,HMV21/000097/1021/224,24.0,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/224,Close,Closed,200131-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...","[-0.5118116, -0.33358994, 0.265683, 0.27577057...","[-0.7624667, -0.5281035, 0.28931808, 0.2974607...","[-0.6633217, -0.45673603, 0.28851366, 0.286182...",NaN,NaN,0.0
1,No,MIREP,HMV21/000097/1021/395,24.0,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,HMV21/000097/1021/395,Close,Closed,200131-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...","[-0.76240206, -0.33975542, 0.09831488, 0.46255...","[-0.79862016, -0.5617091, 0.34904265, 0.431891...","[-0.7762478, -0.49182302, 0.27899, 0.43873632,...",NaN,HMV21/000097/1021/395,1.0
2,No,MIREP,HMV21/000097/1021/790,52.0,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,HMV21/000097/1021/790,Close,Closed,200133-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...","[-0.74351245, -0.6697794, 0.44665995, 0.412511...","[-0.69442767, -0.6254904, 0.55448556, 0.489238...","[-0.6884283, -0.6241946, 0.5414287, 0.47188964...",NaN,NaN,0.0
3,No,MIREP,HMV21/000097/1021/225,24.0,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/225,Close,Closed,200151-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...","[-0.5605606, -0.38250473, 0.14621824, 0.153886...","[-0.7726478, -0.4959983, 0.20929237, 0.2909674...","[-0.6885451, -0.4502917, 0.20185933, 0.2440129...",NaN,NaN,0.0
4,No,MIREP,HMV21/000097/1021/362,24.0,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...",CARRIED OUT REPLACEMENT OF CONNECTOR 117MY-A A...,HMV21/000097/1021/362,Close,Closed,200151-01-1,...,NaN,NaN,NaN,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...","[-0.71904707, 0.024312794, 0.016751565, 0.2885...","[-0.5845081, -0.44859397, 0.22005159, 0.261085...","[-0.5941808, -0.30838522, 0.1675388, 0.2591248...",NaN,HMV21/000097/1021/362,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24027,No,MIREP,HMV21/000138/1221/1228,31.0,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,CARRIED OUT TROUBLESHOOTING OF AIDS DATA NOT S...,HMV21/000138/1221/1228,Close,Closed,AWR21/005357/0322,...,NaN,NaN,NaN,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,"[-0.6396081, -0.2312267, 0.034146167, -0.11750...","[-0.71913266, -0.70069474, 0.58397484, 0.39729...","[-0.70333195, -0.56126887, 0.4623483, 0.261023...",NaN,NaN,0.0
24028,No,MIREP,HMV21/000138/1221/1229,28.0,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...",TROUBLESHOOTING FOR THE FAULT CARRIED OUT AS P...,HMV21/000138/1221/1229,Close,Closed,AWR22/004644/1122,...,NaN,NaN,NaN,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...","[-0.35522807, -0.3229499, 0.024949888, 0.21241...","[-0.6559304, -0.55850506, 0.5316232, 0.5625286...","[-0.59365004, -0.5045403, 0.42566532, 0.497925...",NaN,NaN,0.0
24029,No,MIREP,HMV21/000138/1221/1230,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AND TESTS AS PER S...,HMV21/000138/1221/1230,Close,Closed,AWR22/004750/1122,...,NaN,NaN,NaN,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,"[-0.58108073, -0.36657876, 0.010545301, 0.5320...","[-0.5646659, -0.73799634, 0.5119383, 0.5037746...","[-0.565673, -0.67673814, 0.45982942, 0.4918415...",NaN,NaN,0.0
24030,No,MIREP,HMV21/000138/1221/1231,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AS PER TSM 31-32-0...,HMV21/000138/1221/1231

In [136]:
group_df = group_df.drop_duplicates(subset=['Log Item #']).copy()

# Reset the index of df_group if needed
# df_group.reset_index(drop=True, inplace=True)

# Saving the transformed DataFrame as df_group
group_df.to_csv('./tfidfnew1/final_group.csv', index=True)

In [137]:
group_df

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Reason for Deferral,Auth. Ref. #,Message Center,description_correction,description_embedding,corrective_action_embedding,description_correction_embedding,SourceTask1,obsid_s,group
0,No,MIREP,HMV21/000097/1021/224,24.0,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/224,Close,Closed,200131-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200131-01-1, FOUND C...","[-0.5118116, -0.33358994, 0.265683, 0.27577057...","[-0.7624667, -0.5281035, 0.28931808, 0.2974607...","[-0.6633217, -0.45673603, 0.28851366, 0.286182...",NaN,NaN,0.0
1,No,MIREP,HMV21/000097/1021/395,24.0,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...",REPLACED FWD CARGO DOOR PROXIMITY SENSOR (30WV...,HMV21/000097/1021/395,Close,Closed,200131-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK 200131-01-1, FOUND FWD...","[-0.76240206, -0.33975542, 0.09831488, 0.46255...","[-0.79862016, -0.5617091, 0.34904265, 0.431891...","[-0.7762478, -0.49182302, 0.27899, 0.43873632,...",NaN,HMV21/000097/1021/395,1.0
2,No,MIREP,HMV21/000097/1021/790,52.0,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...",CARRIED OUT REMOVAL OF INDICATOR LIGHT (9MJ)AS...,HMV21/000097/1021/790,Close,Closed,200133-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200133-01-1, FOUND I...","[-0.74351245, -0.6697794, 0.44665995, 0.412511...","[-0.69442767, -0.6254904, 0.55448556, 0.489238...","[-0.6884283, -0.6241946, 0.5414287, 0.47188964...",NaN,NaN,0.0
3,No,MIREP,HMV21/000097/1021/225,24.0,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...",CARRIED OUT REPLACEMENT OF ELBOW OF CONNECTOR ...,HMV21/000097/1021/225,Close,Closed,200151-01-1,...,NaN,NaN,NaN,"WHILE CARRYING OUT TASK : 200151-01-1, FOUND \...","[-0.5605606, -0.38250473, 0.14621824, 0.153886...","[-0.7726478, -0.4959983, 0.20929237, 0.2909674...","[-0.6885451, -0.4502917, 0.20185933, 0.2440129...",NaN,NaN,0.0
4,No,MIREP,HMV21/000097/1021/362,24.0,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...",CARRIED OUT REPLACEMENT OF CONNECTOR 117MY-A A...,HMV21/000097/1021/362,Close,Closed,200151-01-1,...,NaN,NaN,NaN,"AFTER THE REMOVAL OF AFT CARGO FLOOR PANELS, F...","[-0.71904707, 0.024312794, 0.016751565, 0.2885...","[-0.5845081, -0.44859397, 0.22005159, 0.261085...","[-0.5941808, -0.30838522, 0.1675388, 0.2591248...",NaN,HMV21/000097/1021/362,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24027,No,MIREP,HMV21/000138/1221/1228,31.0,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,CARRIED OUT TROUBLESHOOTING OF AIDS DATA NOT S...,HMV21/000138/1221/1228,Close,Closed,AWR21/005357/0322,...,NaN,NaN,NaN,DURING EGR FOUND AIDS DATA NOT SHOWN IN MCDU 1...,"[-0.6396081, -0.2312267, 0.034146167, -0.11750...","[-0.71913266, -0.70069474, 0.58397484, 0.39729...","[-0.70333195, -0.56126887, 0.4623483, 0.261023...",NaN,NaN,0.0
24028,No,MIREP,HMV21/000138/1221/1229,28.0,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...",TROUBLESHOOTING FOR THE FAULT CARRIED OUT AS P...,HMV21/000138/1221/1229,Close,Closed,AWR22/004644/1122,...,NaN,NaN,NaN,"CARRY OUT RECTIFICATION FOR EGR FAULT ""FUEL IN...","[-0.35522807, -0.3229499, 0.024949888, 0.21241...","[-0.6559304, -0.55850506, 0.5316232, 0.5625286...","[-0.59365004, -0.5045403, 0.42566532, 0.497925...",NaN,NaN,0.0
24029,No,MIREP,HMV21/000138/1221/1230,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AND TESTS AS PER S...,HMV21/000138/1221/1230,Close,Closed,AWR22/004750/1122,...,NaN,NaN,NaN,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,"[-0.58108073, -0.36657876, 0.010545301, 0.5320...","[-0.5646659, -0.73799634, 0.5119383, 0.5037746...","[-0.565673, -0.67673814, 0.45982942, 0.4918415...",NaN,NaN,0.0
24030,No,MIREP,HMV21/000138/1221/1231,0.0,CARRY OUT RECTIFICATION OF TEST FLIGHT PFR FAU...,CARRIED OUT TROUBLESHOOTING AS PER TSM 31-32-0...,HMV21/000138/1221/1231

In [138]:
#df_des1 = pd.read_csv('tfidf/cos_sim_desc.csv', index_col=0)

In [139]:
#df_des1

In [140]:
#df_des1= threshold_transform(df_des, threshold=0.4)
#df_des1